In [ ]:
# Date: 2025-12-25
# Bike Rental Forecasting – Clean, single-cell notebook
# -----------------------------------------------------
# This notebook is intentionally a single code cell to make the workflow easy to follow end-to-end.
# It prints intermediate results at each decision point, so it reads like an executable report.
#
# Adjust the paths below to your local files
HOUR_CSV_PATH = r'hour.csv'   # <-- change if needed
DAY_CSV_PATH  = r'day.csv'    # optional; only used for consistency cross-checks

ANALYSE = True               # toggles extra plots & model comparison output
FORECAST_HOURS = 24          # how many future hours to forecast beyond the last observed timestamp
K_MISSING_THRESHOLD = 2      # <=K => human_error; >K => extraordinary_event (with manual overrides)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from pathlib import Path

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.inspection import permutation_importance
from sklearn.ensemble import HistGradientBoostingRegressor

# -----------------------------
# Pretty printing helpers
# -----------------------------
def section(title: str):
    print("\n" + "="*90)
    print(title)
    print("="*90)

def show_df(df: pd.DataFrame, n=8, name=None):
    if name:
        print(f"\n{name} (showing first {n} rows):")
    display(df.head(n)) if "display" in globals() else print(df.head(n))

# -----------------------------
# 1) Load data
# -----------------------------
section("1) Load data")

hour_path = HOUR_CSV_PATH
if not Path(hour_path).exists():
    raise FileNotFoundError(f"Cannot find {HOUR_CSV_PATH}. Set HOUR_CSV_PATH to your local hour.csv path.")

df_hourly = pd.read_csv(hour_path)

print("Hourly shape:", df_hourly.shape)
print("Hourly columns:", list(df_hourly.columns))

# Optional daily file for consistency checks
df_daily = None
day_path = DAY_CSV_PATH
if Path(day_path).exists():
    df_daily = pd.read_csv(day_path)
    print("Daily shape:", df_daily.shape)
else:
    print("Daily file not found (that's ok). DAY_CSV_PATH can be left as-is if you only work on hourly.")

# Ensure types & sorting
df_hourly["dteday"] = pd.to_datetime(df_hourly["dteday"])
df_hourly["hr"] = pd.to_numeric(df_hourly["hr"], errors="coerce").astype(int)
df_hourly = df_hourly.sort_values(["dteday", "hr"]).reset_index(drop=True)

if df_daily is not None:
    df_daily["dteday"] = pd.to_datetime(df_daily["dteday"])
    df_daily = df_daily.sort_values("dteday").reset_index(drop=True)

# -----------------------------
# 2) Core sanity checks
# -----------------------------
section("2) Core sanity checks (identity + duplicates + missing days)")

# cnt identity check
hourly_identity_ok = (df_hourly["casual"] + df_hourly["registered"] == df_hourly["cnt"]).all()
print("Hourly cnt = casual + registered:", hourly_identity_ok)

if df_daily is not None and set(["casual","registered","cnt"]).issubset(df_daily.columns):
    daily_identity_ok = (df_daily["casual"] + df_daily["registered"] == df_daily["cnt"]).all()
    print("Daily cnt = casual + registered:", daily_identity_ok)

# duplicates check
dup = df_hourly.duplicated(subset=["dteday","hr"]).sum()
print("Duplicate (dteday, hr):", dup)

# missing entire days check
full_days = pd.date_range(df_hourly["dteday"].min(), df_hourly["dteday"].max(), freq="D")
present_days = pd.Index(df_hourly["dteday"].unique())
missing_days = full_days.difference(present_days)

print(f"Missing entire days: {len(missing_days)}")
if len(missing_days):
    print(missing_days)

# -----------------------------
# 3) Missing hours per day (gaps) + classification
# -----------------------------
section("3) Missing hours per day and gap classification")

missing_by_day = (
    df_hourly.groupby("dteday")["hr"]
    .apply(lambda x: sorted(set(range(24)) - set(x)))
)
missing_by_day = missing_by_day[missing_by_day.apply(len) > 0]

missing_summary = pd.DataFrame({
    "missing_hours": missing_by_day,
    "n_missing_hours": missing_by_day.apply(len)
}).sort_values("n_missing_hours", ascending=False)

print("Days with missing hours:", len(missing_summary))
print("\nMissing-hour count distribution:")
print(missing_summary["n_missing_hours"].value_counts().sort_index())

if len(missing_summary):
    print("\nTop days by missing hours:")
    print(missing_summary.head(10))

# Manual event mapping (based on your investigation)
EVENTS = {
    "2011-08-27": "Hurricane Irene (evening shutdown)",
    "2011-01-26": "Heavy snowfall",
    "2011-01-27": "Heavy snowfall",
    "2011-01-18": "Protests (Hu Jintao visit)",
    "2012-10-29": "Hurricane Sandy",
    "2012-10-30": "Hurricane Sandy",
}
HUMAN_OVERRIDE = ["2011-02-22"]  # treat as human error even if many hours missing

event_days = pd.to_datetime(list(EVENTS.keys()))
human_override_days = pd.to_datetime(HUMAN_OVERRIDE)

small_gap_days = missing_summary[missing_summary["n_missing_hours"] <= K_MISSING_THRESHOLD].index
large_gap_days = missing_summary[missing_summary["n_missing_hours"] > K_MISSING_THRESHOLD].index

# Apply overrides
large_gap_days = large_gap_days.union(event_days).difference(human_override_days)
small_gap_days = small_gap_days.union(human_override_days).difference(event_days)

print(f"\nHuman error days (<= {K_MISSING_THRESHOLD} missing hours, with overrides): {len(small_gap_days)}")
print(f"Extraordinary event days (> {K_MISSING_THRESHOLD} missing hours, with overrides): {len(large_gap_days)}")

event_table = pd.DataFrame({
    "dteday": event_days,
    "event": [EVENTS[d.strftime("%Y-%m-%d")] for d in event_days]
}).sort_values("dteday")

print("\nExtraordinary event mapping:")
print(event_table.to_string(index=False))

# -----------------------------
# 4) Repair to full 24h/day grid + imputation rules
# -----------------------------
section("4) Repair hourly grid and fill missing values based on gap type")

TARGET_COLS = ["cnt","casual","registered"]
CAL_COLS = ["season","yr","mnth"]
DAY_FLAG_COLS = ["holiday","weekday","workingday"]
WEATHER_COLS = ["weathersit","temp","atemp","hum","windspeed"]

df = df_hourly.copy()

# Ensure numeric for fillable columns
for c in TARGET_COLS + CAL_COLS + DAY_FLAG_COLS + WEATHER_COLS:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Build full day x hour grid
all_days = pd.Index(df["dteday"].unique()).sort_values()
full_index = pd.MultiIndex.from_product([all_days, range(24)], names=["dteday","hr"])

df_full = (
    df.set_index(["dteday","hr"])
      .reindex(full_index)
      .sort_index()
      .reset_index()
)

# Timestamp convenience column
df_full["timestamp"] = df_full["dteday"] + pd.to_timedelta(df_full["hr"], unit="h")

# Assign gap_type for each day
df_full["gap_type"] = "none"
df_full.loc[df_full["dteday"].isin(small_gap_days), "gap_type"] = "human_error"
df_full.loc[df_full["dteday"].isin(large_gap_days), "gap_type"] = "extraordinary_event"

row_missing = df_full["cnt"].isna()
mask_human_missing = (df_full["gap_type"] == "human_error") & row_missing
mask_event_missing = (df_full["gap_type"] == "extraordinary_event") & row_missing

# 4.1 Human error: interpolate within day for targets + weather
for col in TARGET_COLS + WEATHER_COLS:
    df_full.loc[df_full["gap_type"] == "human_error", col] = (
        df_full.loc[df_full["gap_type"] == "human_error"]
        .groupby("dteday")[col]
        .transform(lambda s: s.interpolate(limit_direction="both"))
    )

# 4.2 Extraordinary events:
# - targets for missing rows only -> 0
df_full.loc[mask_event_missing, TARGET_COLS] = 0.0

# - calendar fields should be constant within day -> forward fill within day
for col in CAL_COLS:
    df_full[col] = df_full.groupby("dteday")[col].ffill()

# - recompute weekday + workingday, keep holiday mode if present
def fill_day_flags(group: pd.DataFrame) -> pd.DataFrame:
    d = group.name
    weekday = d.weekday()  # 0=Mon … 6=Sun
    group["weekday"] = weekday

    # if holiday exists and has any value, use mode; else assume 0
    if group["holiday"].notna().any():
        holiday_val = int(group["holiday"].mode().iloc[0])
    else:
        holiday_val = 0
    group["holiday"] = holiday_val
    group["workingday"] = int((weekday < 5) and (holiday_val == 0))
    return group

df_full = df_full.groupby("dteday", group_keys=False).apply(fill_day_flags)

# - weather for extraordinary-event missing rows: mean of nearest valid neighbors in time
df_full = df_full.sort_values(["dteday","hr"]).reset_index(drop=True)
for col in WEATHER_COLS:
    prev_val = df_full[col].ffill()
    next_val = df_full[col].bfill()
    df_full.loc[mask_event_missing, col] = (prev_val[mask_event_missing] + next_val[mask_event_missing]) / 2

# Snap weathersit back to {1,2,3,4} (avoid 3.5 etc.)
if "weathersit" in df_full.columns:
    df_full["weathersit"] = (
        df_full["weathersit"]
        .round()
        .clip(lower=1, upper=4)
        .astype("Int64")
    )

# Enforce cnt identity (after fills)
identity_ok = (df_full["casual"] + df_full["registered"] == df_full["cnt"])
print("Identity cnt = casual + registered holds:", identity_ok.all())

# Remaining NaNs
nan_counts = df_full[TARGET_COLS + CAL_COLS + DAY_FLAG_COLS + WEATHER_COLS].isna().sum().sort_values(ascending=False)
print("\nRemaining NaNs (top):")
print(nan_counts.head(15))

# -----------------------------
# 5) Optional: consistency with daily file (hourly aggregated vs daily)
# -----------------------------
section("5) Optional cross-check: hourly aggregation matches daily file (if available)")

if df_daily is not None:
    hourly_daily = (
        df_full.groupby("dteday")[["casual","registered","cnt"]]
              .sum()
              .reset_index()
    )
    cmp = hourly_daily.merge(df_daily[["dteday","casual","registered","cnt"]],
                             on="dteday", suffixes=("_hourly","_daily"))
    for col in ["casual","registered","cnt"]:
        cmp[f"{col}_equal"] = cmp[f"{col}_hourly"] == cmp[f"{col}_daily"]
    print("All equal:")
    print(cmp[[c for c in cmp.columns if c.endswith("_equal")]].all())
else:
    print("Skipped (daily file not provided).")

# -----------------------------
# 6) EDA: key plots + numeric summaries used for decisions
# -----------------------------
section("6) EDA snapshots (seasonality, weekday patterns, hourly profile, weather correlations)")

# Correlations with cnt (hourly)
corr = df_full[["cnt","temp","atemp","hum","windspeed"]].corr(numeric_only=True)["cnt"].sort_values(ascending=False)
print("Correlation with cnt:")
print(corr)

# workingday/holiday group means
grp = df_full.groupby(["workingday","holiday"])[["cnt","casual","registered"]].mean()
print("\nMean rentals by workingday/holiday:")
print(grp)

if ANALYSE:
    # Seasonal lines on daily totals (from hourly)
    daily_series = df_full.groupby("dteday")[["casual","registered"]].sum().reset_index()
    # Attach season from first row of each day
    day_season = df_full.groupby("dteday")["season"].first().reset_index()
    daily_series = daily_series.merge(day_season, on="dteday", how="left").sort_values("dteday")

    fig, ax = plt.subplots(figsize=(10,4))
    ax.plot(daily_series["dteday"], daily_series["casual"], label="casual")
    ax.plot(daily_series["dteday"], daily_series["registered"], label="registered")
    season_change = daily_series["season"].ne(daily_series["season"].shift())
    for x in daily_series.loc[season_change, "dteday"]:
        ax.axvline(x=x, linestyle="--", alpha=0.35)
    ax.set_xlabel("dteday")
    ax.set_ylabel("Daily count")
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=4))
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
    fig.autofmt_xdate()
    ax.legend()
    ax.set_title("Daily totals (casual vs registered) with season change markers")
    plt.show()

    # Weekday profile (hourly mean)
    by_weekday = df_full.groupby("weekday")[["casual","registered"]].mean().reset_index()
    plt.figure(figsize=(8,3))
    plt.plot(by_weekday["weekday"], by_weekday["casual"], label="casual")
    plt.plot(by_weekday["weekday"], by_weekday["registered"], label="registered")
    plt.xlabel("weekday (0=Mon)")
    plt.ylabel("Avg hourly count")
    plt.title("Average hourly rentals by weekday")
    plt.legend()
    plt.show()

    # Hour-of-day profile
    by_hour = df_full.groupby("hr")[["casual","registered"]].mean().reset_index()
    plt.figure(figsize=(8,3))
    plt.plot(by_hour["hr"], by_hour["casual"], label="casual")
    plt.plot(by_hour["hr"], by_hour["registered"], label="registered")
    plt.xlabel("hour of day")
    plt.ylabel("Avg hourly count")
    plt.xticks(range(0,24))
    plt.title("Average rentals by hour of day (cnt = casual + registered)")
    plt.legend()
    plt.show()

# -----------------------------
# 7) Feature engineering (final set used for modeling)
# -----------------------------
section("7) Feature engineering (final feature set used for modeling)")

TARGET = "cnt"
df_feat = df_full.copy()
df_feat["dteday"] = pd.to_datetime(df_feat["dteday"])

# Lag features
df_feat["lag_24"] = df_feat[TARGET].shift(24)
df_feat["lag_168"] = df_feat[TARGET].shift(168)

# Season dummies
season_map = {1:"spring", 2:"summer", 3:"fall", 4:"winter"}
df_feat["season_name"] = df_feat["season"].map(season_map)
season_dummies = pd.get_dummies(df_feat["season_name"], prefix="season")

# Hour bucket dummies
def hr_to_period(hr):
    if 0 <= hr <= 5: return "night"
    if 6 <= hr <= 11: return "morning"
    if 12 <= hr <= 17: return "afternoon"
    return "evening"

df_feat["hr_period"] = df_feat["hr"].apply(hr_to_period)
hr_dummies = pd.get_dummies(df_feat["hr_period"], prefix="hr")

# Regime features
df_feat["wd_morning"] = ((df_feat["workingday"] == 1) & (df_feat["hr_period"] == "morning")).astype(int)
df_feat["wd_evening"] = ((df_feat["workingday"] == 1) & (df_feat["hr_period"] == "evening")).astype(int)

BEHAVIOR_FEATURES = ["wd_morning","wd_evening","holiday","workingday"]
NUM_FEATURES = ["lag_24","lag_168"]

# Weather buckets (interpretable)
df_feat["atemp_bucket"] = pd.qcut(df_feat["atemp"], q=3, labels=["cold","medium","hot"])
df_feat["hum_bucket"] = pd.qcut(df_feat["hum"], q=3, labels=["not_humid","medium","humid"])
df_feat["windspeed_bucket"] = pd.qcut(df_feat["windspeed"], q=3, labels=["not_windy","medium","windy"])

atemp_dummies = pd.get_dummies(df_feat["atemp_bucket"], prefix="atemp")
hum_dummies = pd.get_dummies(df_feat["hum_bucket"], prefix="hum")
wind_dummies = pd.get_dummies(df_feat["windspeed_bucket"], prefix="wind")

# Weather situation dummies
weathersit_dummies = pd.get_dummies(df_feat["weathersit"].astype("Int64"), prefix="weathersit")

# Final X/y
X = pd.concat(
    [
        df_feat[NUM_FEATURES + BEHAVIOR_FEATURES],
        season_dummies,
        hr_dummies,
        atemp_dummies,
        hum_dummies,
        wind_dummies,
        weathersit_dummies,
    ],
    axis=1
)
y = df_feat[TARGET]

data = pd.concat([X, y], axis=1).dropna()
X = data.drop(columns=[TARGET])
y = data[TARGET]

# Convert booleans to ints
bool_cols = X.select_dtypes(include="bool").columns
X[bool_cols] = X[bool_cols].astype(int)

# Drop one baseline per dummy block to avoid dummy trap
REF_COLS = ["hr_night","season_winter","atemp_medium","hum_medium","wind_medium","weathersit_1"]
X = X.drop(columns=[c for c in REF_COLS if c in X.columns], errors="ignore")

print("Rows after feature prep:", len(X))
print("Features:", len(X.columns))

# -----------------------------
# 8) Time-based train/test split + model comparison
# -----------------------------
section("8) Model comparison (time split, MAE)")

# Time split: last 20% as test
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print("Train rows:", len(X_train), "Test rows:", len(X_test))

# Baseline: seasonal naive using lag_24
# (Needs alignment: y_test corresponds to rows in X_test)
naive_pred = X_test["lag_24"].values
mae_naive = mean_absolute_error(y_test, naive_pred)

# Ridge
ridge = Ridge(alpha=1.0, random_state=0)
ridge.fit(X_train, y_train)
ridge_pred = ridge.predict(X_test)
mae_ridge = mean_absolute_error(y_test, ridge_pred)

# Gradient boosting (final choice)
gbr = HistGradientBoostingRegressor(
    learning_rate=0.05,
    max_depth=6,
    max_iter=400,
    random_state=0
)
gbr.fit(X_train, y_train)
gbr_pred = gbr.predict(X_test)
mae_gbr = mean_absolute_error(y_test, gbr_pred)

results = pd.DataFrame({
    "Model": ["Seasonal Naive (lag_24)", "Ridge Regression", "HistGradientBoostingRegressor"],
    "MAE": [mae_naive, mae_ridge, mae_gbr]
}).sort_values("MAE")

print("\nModel comparison (lower MAE is better):")
print(results.to_string(index=False))

if ANALYSE:
    # Plot a week of predictions vs actual
    n = min(24*7, len(y_test))
    plt.figure(figsize=(10,3))
    plt.plot(y_test.iloc[:n].values, label="actual")
    plt.plot(gbr_pred[:n], label="gbr_pred")
    plt.plot(ridge_pred[:n], label="ridge_pred", alpha=0.7)
    plt.title("Predicted vs actual (first ~week of test set)")
    plt.xlabel("Test hour index")
    plt.ylabel("cnt")
    plt.legend()
    plt.show()

# -----------------------------
# 9) Feature importance sanity check (GBR vs Ridge)
# -----------------------------
section("9) Feature importance comparison (does GBR align with Ridge signals?)")

# Ridge "importance": absolute coefficient
ridge_imp = pd.Series(np.abs(ridge.coef_), index=X.columns).sort_values(ascending=False)

# GBR: permutation importance on a subsample for speed
rng = np.random.default_rng(0)
sub_idx = rng.choice(np.arange(len(X_test)), size=min(2000, len(X_test)), replace=False)
perm = permutation_importance(gbr, X_test.iloc[sub_idx], y_test.iloc[sub_idx],
                              n_repeats=5, random_state=0, scoring="neg_mean_absolute_error")

gbr_imp = pd.Series(perm.importances_mean, index=X.columns).sort_values(ascending=False)

top = 15
imp_compare = pd.DataFrame({
    "ridge_importance": ridge_imp.head(top),
    "gbr_importance": gbr_imp.head(top),
}).fillna(0).sort_values(by="gbr_importance",ascending=False)

print(f"Top {top} features (Ridge abs(coef) and GBR permutation importance):")
print(imp_compare)

# -----------------------------
# 10) Simple forward forecast example (next day / next horizon)
# -----------------------------
section("10) Forward forecast example (recursive, next unseen hours)")

# For forecasting, we extend the df_full timeline. We need the same feature logic.
# We'll generate future rows by hour and fill exogenous variables with simple defaults.
# In a real production system, you'd replace this with weather forecasts and holiday calendars.

df_fc = df_full.copy().sort_values(["dteday","hr"]).reset_index(drop=True)

last_ts = df_fc["timestamp"].max()
future_ts = pd.date_range(last_ts + pd.Timedelta(hours=1), periods=FORECAST_HOURS, freq="H")
future = pd.DataFrame({
    "timestamp": future_ts,
    "dteday": future_ts.floor("D"),
    "hr": future_ts.hour
})

# Fill calendar fields using the last known values by day (simple, robust baseline)
# season/yr/mnth copied from last available day with same month/day context is not possible;
# so we copy from the last observed day (good enough for the assignment demo).
last_day = df_fc["dteday"].max()
last_day_row = df_fc[df_fc["dteday"] == last_day].iloc[-1]

for col in ["season","yr","mnth","weathersit","temp","atemp","hum","windspeed","holiday","workingday","weekday"]:
    if col in df_fc.columns:
        future[col] = last_day_row[col]

# Recompute weekday/workingday from date (prefer correctness)
future["weekday"] = future["dteday"].dt.weekday
# keep holiday default 0 unless you have a calendar feed
future["holiday"] = 0
future["workingday"] = ((future["weekday"] < 5) & (future["holiday"] == 0)).astype(int)

# Append future rows with NaN targets (to be predicted)
for col in TARGET_COLS:
    future[col] = np.nan
df_fc_ext = pd.concat([df_fc, future], ignore_index=True).sort_values("timestamp").reset_index(drop=True)

# Recursive prediction: fill cnt one step at a time
def build_features_frame(df_in: pd.DataFrame) -> pd.DataFrame:
    d = df_in.copy()
    d["dteday"] = pd.to_datetime(d["dteday"])

    d["lag_24"] = d["cnt"].shift(24)
    d["lag_168"] = d["cnt"].shift(168)

    d["season_name"] = d["season"].map(season_map)
    season_dum = pd.get_dummies(d["season_name"], prefix="season")
    d["hr_period"] = d["hr"].apply(hr_to_period)
    hr_dum = pd.get_dummies(d["hr_period"], prefix="hr")

    d["wd_morning"] = ((d["workingday"] == 1) & (d["hr_period"] == "morning")).astype(int)
    d["wd_evening"] = ((d["workingday"] == 1) & (d["hr_period"] == "evening")).astype(int)

    d["atemp_bucket"] = pd.qcut(d["atemp"], q=3, labels=["cold","medium","hot"])
    d["hum_bucket"] = pd.qcut(d["hum"], q=3, labels=["not_humid","medium","humid"])
    d["windspeed_bucket"] = pd.qcut(d["windspeed"], q=3, labels=["not_windy","medium","windy"])

    at_d = pd.get_dummies(d["atemp_bucket"], prefix="atemp")
    hu_d = pd.get_dummies(d["hum_bucket"], prefix="hum")
    wi_d = pd.get_dummies(d["windspeed_bucket"], prefix="wind")
    ws_d = pd.get_dummies(d["weathersit"].astype("Int64"), prefix="weathersit")

    Xf = pd.concat(
        [
            d[NUM_FEATURES + BEHAVIOR_FEATURES],
            season_dum, hr_dum, at_d, hu_d, wi_d, ws_d
        ],
        axis=1
    )

    # align columns to training X
    Xf = Xf.reindex(columns=X.columns, fill_value=0)
    return Xf

# Predict future rows iteratively
df_fc_ext = df_fc_ext.sort_values("timestamp").reset_index(drop=True)
future_idx = df_fc_ext[df_fc_ext["cnt"].isna()].index

for i in future_idx:
    Xf = build_features_frame(df_fc_ext.iloc[:i+1])
    x_last = Xf.iloc[[i]]
    pred = float(gbr.predict(x_last)[0])

    # production guards
    pred = max(0.0, pred)
    pred = round(pred)

    df_fc_ext.loc[i, "cnt"] = pred

# Extract forecast
forecast = df_fc_ext.loc[future_idx, ["timestamp","dteday","hr","cnt"]].copy()
forecast["cnt"] = forecast["cnt"].astype("Int64")

print("Forecast sample:")
print(forecast.head(24).to_string(index=False))

daily_forecast = forecast.groupby("dteday")["cnt"].sum().reset_index(name="cnt_daily_forecast")
print("\nDaily forecast totals:")
print(daily_forecast.to_string(index=False))
